<a href="https://colab.research.google.com/github/HimanshuMK/Neural-Network-from-Scratch/blob/main/FullCodeNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [1]:
import numpy as np

# Creating a Dense Layer
The Layer_Dense class implements a dense (fully connected) layer in a neural network.

1. Initialization (`__init__` method):
  *   Takes the number of inputs (n_inputs) and the number of neurons (n_neurons).
  *   Initializes the weights with small random values and biases with zeros.


2. Forward Pass (`forward` method):
  *   Takes the input data (inputs).
  *   Computes the output by performing a dot product of inputs and weights, then adds biases.
  *   Stores the input values for use in the backward pass.


3. Backward Pass (`backward` method):
  * Takes the gradient of the loss with respect to the output (dvalues).
  * Computes the gradients with respect to weights (dweights), biases (dbiases), and inputs (dinputs).






In [2]:
# Dense layer
class Layer_Dense :
  # Layer initialization
  def __init__ ( self , n_inputs , n_neurons ):
    # Initialize weights and biases
    self.weights = 0.1 * np.random.randn(n_inputs, n_neurons)
    self.biases = np.zeros(( 1 , n_neurons))


  # Forward pass
  def forward ( self , inputs ):
    # Remember input values
    self.inputs = inputs
    # Calculate output values from inputs, weights and biases
    self.output = np.dot(inputs, self.weights) + self.biases

  # Backward pass
  def backward ( self , dvalues ):
    # Gradients on parameters
    self.dweights = np.dot(self.inputs.T, dvalues)
    self.dbiases = np.sum(dvalues, axis = 0 , keepdims = True )
    # Gradient on values
    self.dinputs = np.dot(dvalues, self.weights.T)

# Mean Squared Error Loss and its gradient

1.   Mean Squared Error Loss (`mse_loss` function):
    *   Takes the predicted values (y_pred) and the true values (y_true).
    *   Calculates the Mean Squared Error by computing the average of the squared differences between y_pred and y_true.


2.   Gradient of Mean Squared Error Loss (`mse_loss_gradient` function):
    *   Takes the predicted values (y_pred) and the true values (y_true).
    *   Computes the gradient of the Mean Squared Error with respect to the predictions. This is calculated as 2 * (y_pred - y_true) / y_true.size, which gives the rate of change of the loss with respect to the predictions.



In [3]:
# Mean Squared Error Loss and its gradient
def mse_loss(y_pred, y_true):
    return np.mean((y_pred - y_true) ** 2)

def mse_loss_gradient(y_pred, y_true):
    return 2 * (y_pred - y_true) / y_true.size

# ReLU activation

1.   Forward Pass (`forward` method):
    * Takes the input data (inputs).
    * Computes the output by applying the ReLU function, which sets all negative values to zero and keeps positive values unchanged.
    * Stores the input values for use in the backward pass.
2.   Backward Pass (`backward` method):
    * Takes the gradient of the loss with respect to the output (dvalues).
    * Copies dvalues to self.dinputs.
    * Sets the gradient to zero where the original input values were negative (because the derivative of ReLU is zero for negative inputs).



In [4]:
# ReLU activation
class Activation_ReLU :
  # Forward pass
  def forward ( self , inputs ):
    self.inputs = inputs
    # Calculate output values from inputs
    self.output = np.maximum( 0 , inputs)

  # Backward pass
  def backward ( self , dvalues ):
    # Since we need to modify original variable,
    # we make a copy of values first
    self.dinputs = dvalues.copy()
    # Zero gradient where input values were negative
    self.dinputs[self.inputs <= 0 ] = 0

#Softmax Activation

1.   Forward Pass (`forward` method):
    * Takes the input data (inputs).
    * Computes the exponentiated values of the inputs after subtracting the maximum value in each input for numerical stability.
    * Normalizes these exponentiated values to calculate the probabilities, ensuring that they sum to 1.
    * Stores the output probabilities.
2.   Backward Pass (`backward` method):
    * Takes the gradient of the loss with respect to the output (dvalues).
    * Initializes an array for the gradients with the same shape as dvalues.
    * For each sample in the batch:
        * Reshapes the output to a column vector.
        * Calculates the Jacobian matrix of the softmax function for the sample.
        * Computes the gradient of the loss with respect to the input using the Jacobian matrix and the gradient of the loss with respect to the output.


In [5]:
#Softmax Activation
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        self.inputs = inputs
        # Subtract max value for numerical stability
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        # Normalizing probabilities
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):
        # Creating uninitialized array
        self.dinputs = np.empty_like(dvalues)
        # Enumerating through outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            # Flattening output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output and its gradient
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate sample-wise gradient
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)


# Categorical Cross-Entropy Loss and Gradient

1.   Categorical Cross-Entropy Loss (`categorical_crossentropy_loss` function):
    * Takes the predicted probabilities (y_pred) and the true labels (y_true).
    * Clips the predicted probabilities to avoid division by zero.
    * Calculates the loss by taking the negative logarithm of the predicted probabilities corresponding to the true labels.
    * Returns the average loss over all samples.

2.   Gradient of Categorical Cross-Entropy Loss(`categorical_crossentropy_loss_gradient` function):
    * Takes the predicted probabilities (y_pred) and the true labels (y_true).
    * Clips the predicted probabilities to avoid division by zero.
    * Converts true labels to one-hot encoding if they are not already.
    * Computes the gradient of the loss with respect to the predicted probabilities by taking the negative ratio of the true labels to the predicted probabilities.
    * Normalizes the gradients by the number of samples.
    * Returns the gradients.



In [6]:
#For Softmax Activation
# Categorical Cross-Entropy Loss
def categorical_crossentropy_loss(y_pred, y_true):
    # Clip data to prevent division by 0
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    # Calculate loss
    correct_confidences = y_pred[range(len(y_pred)), y_true]
    return -np.mean(np.log(correct_confidences))


# Gradient of Categorical Cross-Entropy Loss
def categorical_crossentropy_loss_gradient(y_pred, y_true):
    samples = len(y_pred)
    labels = len(y_pred[0])

    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)

    # Initialize the gradient
    gradients = np.zeros_like(y_pred)

    # Convert labels to one-hot if they are not
    if len(y_true.shape) == 1:
        y_true = np.eye(labels)[y_true]  #eye matrix

    # Calculate gradient
    gradients = -y_true / y_pred
    gradients = gradients / samples

    return gradients


#Common SGD Vanilla optimizer
1.   Initialization (`__init__` method):
    * Sets the initial learning rate (learning_rate) and decay rate (decay).
    * Initializes current_learning_rate with the initial learning rate.
    * Initializes the iteration counter (iterations) to zero.
2.   Pre-Update Parameters (`pre_update_params` method):
    * Adjusts the current learning rate based on the decay rate and the number of iterations.
    * The formula used is:
  $$current\_learning\_rate = learning\_rate * \frac{1}{1 + decay * iterations}$$
3.   Update Parameters (`update_params` method):
    * Updates the weights and biases of a layer by subtracting the product of the current learning rate and the gradients (dweights and dbiases).
4.   Post-Update Parameters (`post_update_params` method):
    * Increments the iteration counter by one after the parameters have been updated.



In [7]:
## optimizers
#Common SGD Vanilla optimizer
class Optimizer_SGD :

  # Initialize optimizer - set settings,
  # learning rate of 1 is default for this optimizer
  def __init__ ( self , learning_rate = 1.0, decay = 0.):
    self.learning_rate = learning_rate
    self.current_learning_rate = learning_rate
    self.decay = decay
    self.iterations = 0

  # Call once before any parameter updates
  def pre_update_params ( self ):
    if self.decay:
      self.current_learning_rate = self.learning_rate *( 1. / ( 1. + self.decay * self.iterations))

  # Update parameters
  def update_params ( self , layer ):

    layer.weights += - self.current_learning_rate * layer.dweights
    layer.biases += - self.current_learning_rate * layer.dbiases

  # Call once after any parameter updates
  def post_update_params ( self ):
    self.iterations += 1



# SGD optimizer with momentum and vanilla

1. Initialization (`__init__` method):
  * Sets the initial learning rate (learning_rate), decay rate (decay), and momentum factor (momentum).
  * Initializes current_learning_rate with the initial learning rate.
  * Initializes the iteration counter (iterations) to zero.
2. Pre-Update Parameters (`pre_update_params` method):
  * Adjusts the current learning rate based on the decay rate and the number of iterations.
  * The formula used is:
  $$ current\_learning\_rate = learning\_rate * \frac{1} {(1 + decay * iterations)}.$$
3. Update Parameters (`update_params` method):
  * With Momentum:
    * Checks if the layer has momentum arrays for weights and biases. If not, initializes them to zero.
    * Computes weight and bias updates using the momentum term and the current gradients.
    * Updates the layer's momentum arrays with these new values.
  * Without Momentum:
    * Computes weight and bias updates using only the current gradients.
  * Updates the weights and biases of the layer using the computed updates.
4. Post-Update Parameters (`post_update_params` method):
  * Increments the iteration counter by one after the parameters have been updated.


In [12]:
# SGD optimizer with momentum and vanilla
class Optimizer_SGD:

    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1., decay=0., momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    # Update parameters
    def update_params(self, layer):

        # If we use momentum
        if self.momentum:

            # If layer does not contain momentum arrays, create them
            # filled with zeros
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                # If there is no momentum array for weights
                # The array doesn't exist for biases yet either.
                layer.bias_momentums = np.zeros_like(layer.biases)

            # Build weight updates with momentum - take previous
            # updates multiplied by retain factor and update with
            # current gradients
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates

            # Build bias updates
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates

        # Vanilla SGD updates (as before momentum update)
        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases

        # Update weights and biases using either
        # vanilla or momentum updates
        layer.weights += weight_updates
        layer.biases += bias_updates


    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1


# Adam optimizer
1. Initialization (`__init__` method):
  * Sets initial values for learning rate (`learning_rate`), decay rate (`decay`), epsilon (`epsilon`), and exponential decay rates for the first (`beta_1`) and second (`beta_2`) moment estimates.
  * Initializes current_learning_rate with the initial learning rate and the iteration counter (`iterations`) to zero.
2. Pre-Update Parameters (`pre_update_params` method):
  * Adjusts the current learning rate based on the decay rate and the number of iterations using the formula:
  $$current\_learning\_rate = learning\_rate * \frac {1} {1 + decay * iterations}$$
3. Update Parameters (`update_params` method):
  * Initialization:
    * Checks if the layer has momentum and cache arrays for weights and biases. If not, initializes them to zero.
  * Momentum Update:
    * Updates the momentum terms for weights and biases using the current gradients and the beta_1 parameter.
  * Bias-Corrected Momentum:
    * Computes bias-corrected momentum terms for weights and biases to counteract the initialization bias.
  * Cache Update:
    * Updates the cache terms for weights and biases using the squared current gradients and the beta_2 parameter.
  * Bias-Corrected Cache:
    * Computes bias-corrected cache terms for weights and biases to counteract the initialization bias.
  * Parameter Update:
    * Updates the weights and biases using the corrected momentum and cache terms, normalizing with the square root of the cache terms and adding epsilon for numerical stability.
4. Post-Update Parameters (`post_update_params` method):
  * Increments the iteration counter by one after the parameters have been updated.

In [9]:
# Adam optimizer
class Optimizer_Adam:

    # Initialize optimizer - set settings
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7,
                 beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    # Update parameters
    def update_params(self, layer):

        # If layer does not contain cache arrays,
        # create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Update momentum  with current gradients
        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases

        # Get corrected momentum
        # self.iteration is 0 at first pass
        # and we need to start with 1 here
        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        # Update cache with squared current gradients
        layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2

        layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases**2
        # Get corrected cache
        weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))

        # Vanilla SGD parameter update + normalization
        # with square rooted cache
        layer.weights += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
        layer.biases += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1


## Dense Layer with regularization

The Layer_Dense class implements a dense (fully connected) layer in a neural network, including support for L1 and L2 regularization.
1. Initialization (__init__ method):
  * Takes the number of inputs (n_inputs), number of neurons (n_neurons), and regularization strengths for weights and biases (L1 and L2).
  * Initializes weights with small random values and biases with zeros.
  * Sets the regularization strengths for weights and biases.
2. Forward Pass (forward method):
  * Takes the input data (inputs).
  * Computes the output by performing a dot product of inputs and weights, then adds biases.
  * Stores the input values for use in the backward pass.
3. Backward Pass (backward method):
  * Takes the gradient of the loss with respect to the output (dvalues).
  * Computes the gradients with respect to weights (dweights) and biases (dbiases).
  * Adds the regularization gradients to dweights and dbiases:
    * L1 Regularization: Adds the sign of the weights or biases multiplied by the L1 regularization strength.
    * L2 Regularization: Adds twice the weights or biases multiplied by the L2 regularization strength.
  * Computes the gradient of the loss with respect to the input values (dinputs).


In [10]:
# Dense layer
class Layer_Dense:

    # Layer initialization
    def __init__(self, n_inputs, n_neurons,
                 weight_regularizer_l1=0, weight_regularizer_l2=0,
                 bias_regularizer_l1=0, bias_regularizer_l2=0):
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        # Set regularization strength
        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2

    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from inputs, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases
    # Backward pass
    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)

        # Gradients on regularization
        # L1 on weights
        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1
        # L2 on weights
        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * self.weights
        # L1 on biases
        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1
        # L2 on biases
        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * self.biases

        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)


## Regularization Loss
The regularization_loss function calculates the total regularization loss for a given layer, incorporating both L1 and L2 regularization for weights and biases.
1. Initialization:
  * Initializes regularization_loss to 0.
2. L1 Regularization on Weights:
  * Adds the L1 regularization loss for weights if weight_regularizer_l1 is greater than 0.
  * The L1 loss is calculated as the product of weight_regularizer_l1 and the sum of the absolute values of the weights.
3. L2 Regularization on Weights:
  * Adds the L2 regularization loss for weights if weight_regularizer_l2 is greater than 0.
  * The L2 loss is calculated as the product of weight_regularizer_l2 and the sum of the squared weights.
4. L1 Regularization on Biases:
  * Adds the L1 regularization loss for biases if bias_regularizer_l1 is greater than 0.
  * The L1 loss is calculated as the product of bias_regularizer_l1 and the sum of the absolute values of the biases.
5. L2 Regularization on Biases:
  * Adds the L2 regularization loss for biases if bias_regularizer_l2 is greater than 0.
  * The L2 loss is calculated as the product of bias_regularizer_l2 and the sum of the squared biases.
6. Return Total Regularization Loss:
  * Returns the total regularization loss computed from the above steps.

In [13]:
# Regularization loss calculation
def regularization_loss(layer):
    # 0 by default
    regularization_loss = 0
    # L1 regularization - weights
    # calculate only when factor greater than 0
    if layer.weight_regularizer_l1 > 0 :
      regularization_loss += layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))
    # L2 regularization - weights
    if layer.weight_regularizer_l2 > 0 :
      regularization_loss += layer.weight_regularizer_l2 * np.sum(layer.weights * layer.weights)

    # L1 regularization - biases
    # calculate only when factor greater than 0
    if layer.bias_regularizer_l1 > 0 :
      regularization_loss += layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))
    # L2 regularization - biases
    if layer.bias_regularizer_l2 > 0 :
      regularization_loss += layer.bias_regularizer_l2 * np.sum(layer.biases * layer.biases)
    return regularization_loss